# Self-correcting RAG with an Agent
_Authored by: [Aymeric Roucher](https://huggingface.co/m-ric)_

> This tutorial is advanced. You should have notions from [this other cookbook](advanced_rag) first!

> Reminder: Retrieval-Augmented-Generation (RAG) is “using an LLM to answer a user query, but basing the answer on information retrieved from a knowledge base”. It has many advantages over using a vanilla or fine-tuned LLM: to name a few, it allows to ground the answer on true facts and reduce confabulations, it allows to provide the LLM with domain-specific knowledge, and it allows fine-grained control of access to information from the knowledge base.

But vanilla RAG has limitations:
- It **performs only one retrieval step** of retrieval**: if the results are bad, the generation in turn will be bad.
- **Documents are retrieved or not based on semantic similarity with the user query**: but for instance, if the user query is a question and the document containing the true answer is in affirmative voice, its similarity score will be downgraded compared to other questions, so it might not be ranked high enough to be retrieved.

Many frameworks have been proposed to alleviate these problems, with ideas in this vein:
- Instead of directly using the user query in semantic search, let the LLM formulate a sentence closer to the documents ([HyDE](https://huggingface.co/papers/2212.10496))
- Critique the generated snippets and re-retrieve if needed as in [Self-Query](https://docs.llamaindex.ai/en/stable/examples/evaluation/RetryQuery/)

But we can recover both these by making a **RAG agent: very simply, an agent armed with a retriever tool!**

This agent will: ✅ Formulate the query itself and ✅ Critique to re-retrieve if needed. So it should naively recover some advanced RAG techniques!

Let's setup this system. 

Run the line below to install required dependancies:

In [1]:
!pip install pandas langchain sentence-transformers faiss-cpu "transformers[agents]"

We first load a knowledge base on which we want to perform RAG: this dataset is a compilation of the documentation pages for many `huggingface` packages, stored as markdown.

In [2]:
import datasets

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

Now we prepare the knowledge base by processing the dataset and storing it into a vector database to be used by the retriever. We are going to use LangChain, since it features excellent utilities for vector databases:

In [3]:
from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from tqdm import tqdm

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained("thenlper/gte-small"),
    chunk_size=200,
    chunk_overlap=20,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

# Split docs and keep only unique ones
print("Splitting documents...")
docs_processed = []
unique_texts = {}
for doc in tqdm(source_docs):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed.append(new_doc)

print(
    "Embedding documents... This should take a few minutes (5 minutes on my MacBook with M1 Pro, yes I'm GPU-quite-rich)"
)
embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")
vectordb = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.COSINE,
)

Splitting documents...


100%|██████████| 2647/2647 [00:34<00:00, 75.77it/s] 


Embedding documents... This should take a few minutes (5 minutes on my MacBook with M1 Pro, yes I'm GPU-quite-rich)


Now that we have the database ready, let’s build a RAG system that answers user queries based on it!

We want our system to select only from the most relevant sources of information, depending on the query.

👉 Let us build our RAG system as an agent that will be free to choose its query and can iterate on the retrieval results!

In [4]:
import json
from transformers.agents import Tool
from langchain_core.vectorstores import VectorStore


class RetrieverTool(Tool):
    name = "retriever"
    description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
    inputs = {
        "query": {
            "type": "text",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "text"

    def __init__(self, vectordb: VectorStore, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb

    def forward(self, query: str, number_of_documents=7) -> str:
        assert isinstance(query, str), "Your search query must be a string"
        number_of_documents = max(int(number_of_documents), 5)

        docs = self.vectordb.similarity_search(
            query,
            k=7,
        )

        return "\nRetrieved documents:\n" + "".join(
            [
                f"===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

Now it’s straightforward to create an agent that leverages this tool!

The agent will need these arguments upon initialization:
- *`tools`*: a list of tools that the agent will be able to call.
- *`llm_engine`*: the LLM that powers the agent.

Our `llm_engine` must be a callable that takes as input a list of [messages](https://huggingface.co/docs/transformers/main/chat_templating) and returns text. It also needs to accept a `stop_sequences` argument that indicates when to stop its generation. For convenience, we directly use the `HfEngine` class provided in the package to get a LLM engine that calls our [Inference API](https://huggingface.co/docs/api-inference/en/index).

In [5]:
from transformers.agents import HfEngine, ReactJsonAgent

llm_engine = HfEngine("CohereForAI/c4ai-command-r-plus")

retriever_tool = RetrieverTool(vectordb)
agent = ReactJsonAgent(
    tools=[retriever_tool], llm_engine=llm_engine, max_iterations=4, verbose=2
)

Since we initialized the agent as a `ReactJsonAgent`, it has been automatically given a default system prompt that tells the LLM engine to process step-by-step and generate tool calls as JSON blobs (you could replace this prompt template with your own as needed).

Then when its `.run()` method is launched, the agent takes care of calling the LLM engine, parsing the tool call JSON blobs and executing these tool calls, all in a loop that ends only when the final answer is provided.

In [8]:
agent_output = agent.run(
    "What is the task performed by the `roberta-large-mnli` checkpoint?"
)

print("Final output:")
print(agent_output)

======== New task ========
What is the task performed by the `roberta-large-mnli` checkpoint?
System prompt is as follows:
You are an expert assistant who can solve any task using JSON tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to the following tools: 'retriever', 'final_answer'
The way you use the tools is by specifying a json blob, ending with '<end_action>'.
Specifically, this json should have an `action` key (name of the tool to use) and an `action_input` key (input to the tool).

The $ACTION_JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. It should be formatted in json. Do not try to escape special characters. Here is the template of a valid $ACTION_JSON_BLOB:
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}<end_action>

Make sure to have the $INPUT as a dictionary in the right format for the tool you are using, and do not put variable names as input if you can find the right valu

Final output:
The roberta-large-mnli checkpoint performs a classification task.


## Evaluate with LLM judge

Does the agent setup improve the performance of our RAG system? Well, let's measure it.

We'll compare the score of our agent system with the different RAG systems tested in [this other cookbook](advanced_rag).

So we take care to use the same evaluation setup, with GPT-4-turbo as a judge for reproducibility. But nowadays I'd just be using [Llama-3](https://huggingface.co/meta-llama/Meta-Llama-3-70B-Instruct).

In [23]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

In [24]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from openai import OpenAI
import os

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

Before running the test let's make the agent less verbose.

In [ ]:
import logging

agent.logger.setLevel(logging.WARNING)

In [79]:
from transformers.agents.agents import AgentError

outputs = []


def serialize_agent_error(obj):
    if isinstance(obj, AgentError):
        return {"error_type": obj.__class__.__name__, "message": obj.message}
    else:
        return str(obj)


for example in tqdm(eval_dataset):
    question = example["question"]
    if question in [el["question"] for el in outputs]:
        continue

    enhanced_question = f"""Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If you cannot find information, do not give up and try calling your retriever again with different arguments!

Question:
{question}"""
    answer = agent.run(enhanced_question)
    print("=======================================================")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f'True answer: {example["answer"]}')

    result = {
        "question": question,
        "true_answer": example["answer"],
        "source_doc": example["source_doc"],
        "generated_answer": answer,
    }
    outputs.append(result)

    with open("outputs.jsonl", "w") as f:
        for d in outputs:
            json.dump(d, f, default=serialize_agent_error)
            f.write("\n")  # add a newline for JSONL format

  0%|          | 0/65 [00:00<?, ?it/s]======== New task ========
Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If you cannot find information, do not give up and try calling your retriever again with different arguments!

Question:
What architecture is the `tokenizers-linux-x64-musl` binary designed for?

Calling tool: 'retriever' with arguments: {'query': 'What architecture is the `tokenizers-linux-x64-musl` binary designed for?'}
Calling tool: 'final_answer' with arguments: {'answer': 'The `tokenizers-linux-x64-musl` binary is designed for the **x86_64-unknown-linux-musl** architecture (Document 0).'}
  2%|▏         | 1/65 [00:15<16:01, 15.02s/it]======== New task ========
Using the information contained in your knowledge base, w

Question: What architecture is the `tokenizers-linux-x64-musl` binary designed for?

Answer: The `tokenizers-linux-x64-musl` binary is designed for the **x86_64-unknown-linux-musl** architecture (Document 0).
True answer: x86_64-unknown-linux-musl


Calling tool: 'retriever' with arguments: {'query': 'BLIP-Diffusion model'}
Calling tool: 'final_answer' with arguments: {'answer': 'The BLIP-Diffusion model is a pre-trained subject representation for controllable text-to-image generation and editing. It enables zero-shot subject-driven generation and control-guided zero-shot generation.'}
  3%|▎         | 2/65 [00:25<12:44, 12.13s/it]======== New task ========
Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If you cannot find information, do not give up and try calling your retriever again with different arguments!

Question:
How can a user claim authorship of a paper on the Hugging Face Hub?



Question: What is the purpose of the BLIP-Diffusion model?

Answer: The BLIP-Diffusion model is a pre-trained subject representation for controllable text-to-image generation and editing. It enables zero-shot subject-driven generation and control-guided zero-shot generation.
True answer: The BLIP-Diffusion model is designed for controllable text-to-image generation and editing.


Could not parse the given action: JSON is invalid: you probably tried to provide multiple tool calls in one action. PROVIDE ONLY ONE TOOL CALL..
Traceback (most recent call last):
  File "/Users/aymeric/Documents/Code/original_transformers/transformers/src/transformers/agents/agents.py", line 83, in parse_json_blob
    json_data = json.loads(json_blob, strict=False)
  File "/opt/homebrew/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10/json/__init__.py", line 359, in loads
    return cls(**kw).decode(s)
  File "/opt/homebrew/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10/json/decoder.py", line 340, in decode
    raise JSONDecodeError("Extra data", s, end)
json.decoder.JSONDecodeError: Extra data: line 6 column 6 (char 165)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/aymeric/Documents/Code/original_transformers/transformers/src/transformers/

...

Question: What is the name of the open-source library created by Hugging Face to simplify Transformer acceleration?

Answer: The name of the open-source library created by Hugging Face to simplify Transformer acceleration is Optimum (also referred to as '🤗 Optimum' and 'Optimum Intel'). This information is derived from multiple documents in the knowledge base.
True answer: Optimum


Calling tool: 'retriever' with arguments: {'query': 'What parameter is used to ensure that elements in a row have the same height in Gradio?'}
Calling tool: 'final_answer' with arguments: {'answer': 'The `equal_height` parameter is used to ensure that elements in a row have the same height in Gradio.'}
 98%|█████████▊| 64/65 [15:24<00:12, 12.17s/it]======== New task ========
Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If you cannot find information, do not give up and try calling your retriever again with different arguments!

Question:
What is the command to install the latest version of Optimum with OpenVINO support?



Question: What parameter is used to ensure that elements in a row have the same height in Gradio?

Answer: The `equal_height` parameter is used to ensure that elements in a row have the same height in Gradio.
True answer: equal_height


Calling tool: 'retriever' with arguments: {'query': 'Optimum installation command with OpenVINO support'}
Calling tool: 'final_answer' with arguments: {'answer': "The command to install the latest version of Optimum with OpenVINO support is:\n\nbash\npip install --upgrade-strategy eager optimum['openvino']\n\n\nThis command ensures that the latest version of Optimum is installed with support for OpenVINO, enabling inference on a variety of Intel processors."}
100%|██████████| 65/65 [15:49<00:00, 14.60s/it]

Question: What is the command to install the latest version of Optimum with OpenVINO support?

Answer: The command to install the latest version of Optimum with OpenVINO support is:

bash
pip install --upgrade-strategy eager optimum['openvino']


This command ensures that the latest version of Optimum is installed with support for OpenVINO, enabling inference on a variety of Intel processors.
True answer: pip install --upgrade-strategy eager optimum["openvino"]


In [81]:
for experiment in tqdm(outputs):
    eval_prompt = EVALUATION_PROMPT.format(
        instruction=experiment["question"],
        response=experiment["generated_answer"],
        reference_answer=experiment["true_answer"],
    )
    messages = [
        {"role": "system", "content": "You are a fair evaluator language model."},
        {"role": "user", "content": eval_prompt},
    ]

    eval_result = (
        client.chat.completions.create(
            model="gpt-4-1106-preview", messages=messages, temperature=0.5
        )
        .choices[0]
        .message.content
    )

    feedback, score = [item.strip() for item in eval_result.split("[RESULT]")]
    experiment[f"eval_score_LLM_judge"] = score
    experiment[f"eval_feedback_LLM_judge"] = feedback

Loaded 65 answers.


100%|██████████| 65/65 [05:02<00:00,  4.65s/it]


In [82]:
import pandas as pd

result = pd.DataFrame.from_dict(outputs)
result = result.loc[~result["generated_answer"].str.contains("Error")]
result["eval_score_LLM_judge_int"] = result["eval_score_LLM_judge"].apply(
    lambda x: int(x)
)
result["eval_score_LLM_judge_int"] = (result["eval_score_LLM_judge_int"] - 1) / 4

print(f"Average score: {result['eval_score_LLM_judge_int'].mean():.3f}")

Average score: 0.850


So we reach 85% score with this simple setup!